Constructs an URL from a list of tile numbers and dates and downloads zipped tiffs into a desired directory. Assuming the corresponding geoportal has open download URLS. Used geoportals: https://www.geoportal.lt/geoportal/ and https://geoportaal.maaamet.ee/eng/

In [ ]:
import time
import requests
from itertools import product


TILE_NUMBERS = [""] # Replace with tile numbers to download
DATES = [""]  # Replace with actual dates from geoportal

# Base geoportal URL format 
BASE_URL = ""

def download_file(url, filename):
    try:
        response = requests.get(url, stream=True)
        response.raise_for_status()
        
        with open(filename, 'wb') as file:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)
        
        print(f"Downloaded: {filename}")
    except requests.RequestException as e:
        print(f"Failed to download {filename}: {e}")

# Download all tile and date combinations provided in lists
for tile, date in product(TILE_NUMBERS, DATES):
    url = BASE_URL.format(tile=tile, date=date)
    filename = f"{tile}_OF_RGB_GeoTIFF_{date}.zip"
    print(f"Fetching: {url}")
    download_file(url, filename)

print("Done")


Processes a directory of tiff files, checks if they have the three RGB bands and outputs processed tiffs with only the RGB bands and JPEG compression (85% quality)

In [ ]:
import os
import rasterio
import numpy as np
from rasterio.enums import Compression


input_dir = ""  # Input directory with raw tiff files
output_dir = ""  # Output directory with processed tiff files

os.makedirs(output_dir, exist_ok=True)

for filename in os.listdir(input_dir):
    if filename.endswith(".tif") or filename.endswith(".tiff"):
        input_filepath = os.path.join(input_dir, filename)
        output_filepath = os.path.join(output_dir, f"RGB_{filename}")

        with rasterio.open(input_filepath) as src:
            if src.count < 3:
                print(f"Skipping {filename}: not enough bands.")
                continue
            
            rgb_data = src.read([1, 2, 3])

            profile = src.profile
            profile.update(
                count=3,  
                compress="JPEG",  
                photometric="YCBCR",  
                driver="GTiff", 
                quality=85,  
                interleave="pixel",  
                tiled=True,
                blockxsize=256,  
                blockysize=256 
            )

            with rasterio.open(output_filepath, "w", **profile) as dst:
                dst.write(rgb_data)

        print(f"Processed {filename} -> {output_filepath}")

print("Done")


Processes RGB tiff files in chunks without dumping the entire directory into RAM and merges the processed inputs into one final tiff image

In [ ]:
import os
import rasterio
import numpy as np
import rasterio.windows as windows


input_dir = ""  
output_dir = ""  

tif_files = [os.path.join(input_dir, f) for f in os.listdir(input_dir) if f.endswith(".tif") or f.endswith(".tiff")]

if not tif_files:
    print("No tiff files found")
    exit()

with rasterio.open(tif_files[0]) as src:
    meta = src.profile.copy()
    dtype = src.dtypes[0]  
    crs = src.crs 

    min_x, min_y, max_x, max_y = src.bounds

# Processing extent
for tif in tif_files:
    with rasterio.open(tif) as src:
        min_x = min(min_x, src.bounds.left)
        min_y = min(min_y, src.bounds.bottom)
        max_x = max(max_x, src.bounds.right)
        max_y = max(max_y, src.bounds.top)

out_transform = rasterio.transform.from_bounds(min_x, min_y, max_x, max_y, meta['width'], meta['height'])

meta.update({
    "driver": "GTiff",  
    "BIGTIFF": "YES", 
    "compress": "JPEG", 
    "photometric": "YCBCR", 
    "interleave": "pixel", 
    "tiled": True, 
    "blockxsize": 256, 
    "blockysize": 256,  
    "quality": 85, 
    "transform": out_transform,
    "crs": crs
})

with rasterio.open(output_dir, "w", **meta) as dest:
    tile_size = 256 

    for tif in tif_files:
        with rasterio.open(tif) as src:
            for j in range(0, src.height, tile_size): 
                for i in range(0, src.width, tile_size): 
                    window = windows.Window(i, j, min(tile_size, src.width - i), min(tile_size, src.height - j))                  
                    data_chunk = src.read(window=window)
                    out_window = windows.Window(i, j, data_chunk.shape[2], data_chunk.shape[1])
                    dest.write(data_chunk, window=out_window)

print(f"Tiff saved at {output_filepath}")


Processes a directory of tiff images, defines their current CRS and reprojects them to a desired CRS (in this case WGS 84: 4326)

In [ ]:
import os
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling


def reproject_tiff(input_folder):
    output_folder = os.path.join(input_folder, "reprojected")
    os.makedirs(output_folder, exist_ok=True)
    
    for filename in os.listdir(input_folder):
        if filename.endswith(".tif") or filename.endswith(".tiff"):
            input_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, filename)

            with rasterio.open(input_path) as src:
                if src.crs is None:
                    print(f"Skipping {filename}: No CRS found.")
                    continue

                dst_crs = 'EPSG:4326'
                transform, width, height = calculate_default_transform(
                    src.crs, dst_crs, src.width, src.height, *src.bounds
                )
                
                kwargs = src.meta.copy()
                kwargs.update({
                    'crs': dst_crs,
                    'transform': transform,
                    'width': width,
                    'height': height,
                    'driver': 'GTiff', 
                    'compress': 'JPEG',  
                    'jpeg_quality': 85, 
                    'photometric': 'YCBCR' 
                })

                with rasterio.open(output_path, 'w', **kwargs) as dst:
                    for i in range(1, src.count + 1):
                        reproject(
                            source=rasterio.band(src, i),
                            destination=rasterio.band(dst, i),
                            src_transform=src.transform,
                            src_crs=src.crs,
                            dst_transform=transform,
                            dst_crs=dst_crs,
                            resampling=Resampling.nearest
                        )
            
            print(f"Reprojected: {filename} → {output_path}")

